In [ ]:
# importing this library so the seed stays the same
import os
import numpy as np

from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample, split_train_in_two_percentage_user_wise
import Utils.not_random
from Data.RecSys2022 import RecSys2022, RecSys2022URMType
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
    split_train_in_two_percentage_user_wise)
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from hybrid import Hybrid

# building the dataset
# later we can call the build method to get different URM
dataset = RecSys2022()
dataset.build(type=RecSys2022URMType.ONE_INTERACTED)

urm = dataset.get_urm()
icm = dataset.get_icm()

urm_train_validation, urm_test = split_train_in_two_percentage_global_sample(
    urm, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

recommender_class = Hybrid

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_test)

hyperparameters_range_dictionary = {
    "epsilon": Real(0.3, 0.8),
    "interactions_count": Integer(3, 60),
    "norm": Categorical([1, 2]),
}

recommender_input_args = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[urm_train, icm],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[urm_train_validation, icm],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

output_folder_path = "result_experiments_5/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 10  # using 10 as an example
n_random_starts = n_cases
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

hyperparameterSearch.search(recommender_input_args,
                            recommender_input_args_last_test=recommender_input_args_last_test,
                            hyperparameter_search_space=hyperparameters_range_dictionary,
                            n_cases=n_cases,
                            n_random_starts=n_random_starts,
                            save_model="last",
                            output_folder_path=output_folder_path,  # Where to save the results
                            output_file_name_root=recommender_class.RECOMMENDER_NAME,  # How to call the files
                            metric_to_optimize=metric_to_optimize,
                            cutoff_to_optimize=cutoff_to_optimize,
                            )


In [13]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,epsilon,interactions_count,norm
0,0.311148,15,2
1,0.402951,7,2
2,0.341381,38,2
3,0.686355,22,2
4,0.345582,59,1
5,0.697721,43,1
6,0.304132,45,2
7,0.422244,40,2
8,0.749316,41,2
9,0.542156,41,1


In [14]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.054547,0.085447,0.078519,0.026114,0.040322,0.171922,0.082667,0.064374,0.354922,0.210473,...,0.982536,0.348723,0.982536,0.042086,8.925163,0.993728,0.0848,0.679514,3.690195,0.404299
1,10,0.054684,0.08573,0.078837,0.02618,0.040462,0.17263,0.082948,0.064576,0.356266,0.211182,...,0.982536,0.350044,0.982536,0.042406,8.910613,0.993631,0.085445,0.678406,3.707005,0.404117
2,10,0.05408,0.084573,0.077674,0.026005,0.040163,0.171818,0.082218,0.063765,0.352868,0.209968,...,0.982536,0.346705,0.982536,0.041949,8.927693,0.993656,0.084523,0.679707,3.719722,0.403977
3,10,0.054467,0.085469,0.07865,0.026126,0.040455,0.172834,0.082891,0.064362,0.356462,0.211197,...,0.982536,0.350237,0.982536,0.043519,8.910227,0.993499,0.087687,0.678377,3.736598,0.404075
4,10,0.054442,0.085302,0.078414,0.026109,0.040376,0.17222,0.082697,0.064266,0.355019,0.210681,...,0.982536,0.348819,0.982536,0.042692,8.926025,0.993641,0.086021,0.67958,3.714783,0.404209
5,10,0.054741,0.086019,0.079146,0.026214,0.040541,0.172875,0.083145,0.064719,0.357635,0.211511,...,0.982536,0.35139,0.982536,0.051859,9.011913,0.993643,0.10449,0.686119,3.672507,0.407219
6,10,0.053958,0.084326,0.077432,0.02591,0.040019,0.171436,0.081996,0.063598,0.352526,0.2094,...,0.982536,0.346369,0.982536,0.041947,8.929322,0.993634,0.08452,0.679831,3.727225,0.403919
7,10,0.054271,0.084823,0.077947,0.026066,0.04026,0.172195,0.082447,0.063989,0.354115,0.210486,...,0.982536,0.347931,0.982536,0.041997,8.919665,0.993637,0.084619,0.679095,3.719359,0.403925
8,10,0.054574,0.085759,0.078951,0.026129,0.040446,0.172788,0.082969,0.064538,0.357,0.211159,...,0.982536,0.350765,0.982536,0.045454,8.931777,0.993577,0.091585,0.680017,3.701996,0.405044


In [15]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'epsilon': 0.5421561572338705, 'interactions_count': 41, 'norm': 1}

In [16]:
import pandas as pd

pd.merge(search_metadata["hyperparameters_df"], pd.Series(pd.Series(search_metadata["result_on_validation_df"]["MAP"]).reset_index()["MAP"]), left_index=True, right_index=True).sort_values(by=["MAP"], ascending=False).head(30)

,epsilon,interactions_count,norm,MAP
9,0.542156,41,1,0.0263
5,0.697721,43,1,0.026214
1,0.402951,7,2,0.02618
8,0.749316,41,2,0.026129
3,0.686355,22,2,0.026126
0,0.311148,15,2,0.026114
4,0.345582,59,1,0.026109
7,0.422244,40,2,0.026066
2,0.341381,38,2,0.026005
6,0.304132,45,2,0.02591


In [ ]:
recommender_final = Hybrid(urm_train_validation, icm)

recommender_final.fit(**best_hyperparameters)

result_df, _ = evaluator_test.evaluateRecommender(recommender_final)
print(f'map@10: {result_df.loc[10]["MAP"]:.7f}')